# Predicting lung cancer survival time by OWKIN

### Problem

- supervised survival prediction problem
- predict the survival time of a patient (remaining days to live) from one three-dimensional CT scan (grayscale image) and a set of pre-extracted quantitative imaging features, as well as clinical data

### Import

In [23]:
import numpy as np
import os
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import autosklearn.regression
import time

# concordance index (C-index)
from metrics_t9gbvr2 import cindex

### Data

- x_train : data_Q0G7b5t
- y_train : output_VSVxRFU.csv
- x_test : data_9Cbe5hx

In [24]:
data_folder_path = "../data"
training_folder_path = os.path.join(data_folder_path, "data_Q0G7b5t")
test_folder_path = os.path.join(data_folder_path, "data_9Cbe5hx")

training_ct_scan_names = [os.path.join(root,file_name) for root,_,file_names in os.walk(training_folder_path) for file_name in file_names if file_name.endswith('.npz')]
test_ct_scan_names = [os.path.join(root,file_name) for root,_,file_names in os.walk(test_folder_path) for file_name in file_names if file_name.endswith('.npz')]

print("Number of training ct scans : {}".format(len(training_ct_scan_names)))
print("Number of test ct scans : {}".format(len(test_ct_scan_names)))

training_features_path = os.path.join(training_folder_path, "features")
test_features_path = os.path.join(test_folder_path, "features")

submission_file_path = "../random_submission_example"

Number of training ct scans : 300
Number of test ct scans : 125


In [25]:
archive = np.load(training_ct_scan_names[0])
scan = archive['scan']
mask = archive['mask']
# scan.shape equals mask.shape

In [26]:
df_train_output = pd.read_csv(os.path.join(data_folder_path, "output_VSVxRFU.csv"), index_col=0)
p0 = df_train_output.loc[202]
print("p0.Event", p0.Event) # prints 1 or 0
print("p0.SurvivalTime", p0.SurvivalTime)
# prints time to event (time to death or time to last known alive) in days

p0.Event 0
p0.SurvivalTime 1378


In [27]:
df_train_output.sample(5)

,SurvivalTime,Event
PatientID,,
328,182,1
16,316,0
33,515,1
244,1369,1
336,524,0


### Interpretation

(`1=death observed`, `0=escaped from study`)

### Load training data

In [28]:
file_name = os.path.join(training_features_path, "clinical_data.csv")
df_training_clinical_data = pd.read_csv(file_name, delimiter=',')
print("Nb rows in df_training_clinical_data : {}".format(len(df_training_clinical_data)))

file_name = os.path.join(training_features_path, "radiomics.csv")
df_training_radiomics = pd.read_csv(file_name, delimiter=',', skiprows=[0,2], header=[0])
df_training_radiomics.rename(columns={'Unnamed: 0': 'PatientID'}, inplace=True)
print("Nb rows in df_training_radiomics : {}".format(len(df_training_radiomics)))

Nb rows in df_training_clinical_data : 300
Nb rows in df_training_radiomics : 300


### Load test data

In [29]:
file_name = os.path.join(test_features_path, "clinical_data.csv")
df_test_clinical_data = pd.read_csv(file_name, delimiter=',')
print("Nb rows in df_training_clinical_data : {}".format(len(df_test_clinical_data)))

file_name = os.path.join(test_features_path, "radiomics.csv")
df_test_radiomics = pd.read_csv(file_name, delimiter=',', skiprows=[0,2], header=[0])
df_test_radiomics.rename(columns={'Unnamed: 0': 'PatientID'}, inplace=True)
print("Nb rows in df_training_radiomics : {}".format(len(df_test_clinical_data)))

Nb rows in df_training_clinical_data : 125
Nb rows in df_training_radiomics : 125


### clinical_data.csv

In [30]:
df_training_clinical_data.sample(5)

,PatientID,Histology,Mstage,Nstage,SourceDataset,Tstage,age
183,26,squamous cell carcinoma,0,2,l1,4,62.4011
232,320,large cell,0,0,l1,4,79.1020
229,330,squamous cell carcinoma,0,0,l1,4,66.0534
155,196,large cell,0,0,l1,1,76.4846
127,51,squamous cell carcinoma,0,0,l1,4,78.6886


#### Are there NaN values in df_training_clinical_data ?

In [31]:
#df_training_clinical_data.info()
df_training_clinical_data.isnull().sum()

PatientID         0
Histology        20
Mstage            0
Nstage            0
SourceDataset     0
Tstage            0
age              16
dtype: int64

### Remark

There are NaN values in columns Histology and age. We will not use these in our study so no problem.

### radiomics.csv

In [32]:
df_training_radiomics.sample(5)

,PatientID,original_shape_Compactness1,original_shape_Compactness2,original_shape_Maximum3DDiameter,original_shape_SphericalDisproportion,original_shape_Sphericity,original_shape_SurfaceArea,original_shape_SurfaceVolumeRatio,original_shape_VoxelVolume,original_firstorder_Energy,...,original_glrlm_LongRunEmphasis,original_glrlm_GrayLevelNonUniformity,original_glrlm_RunLengthNonUniformity,original_glrlm_RunPercentage,original_glrlm_LowGrayLevelRunEmphasis,original_glrlm_HighGrayLevelRunEmphasis,original_glrlm_ShortRunLowGrayLevelEmphasis,original_glrlm_ShortRunHighGrayLevelEmphasis,original_glrlm_LongRunLowGrayLevelEmphasis,original_glrlm_LongRunHighGrayLevelEmphasis
245,307,0.026776,0.254741,93.994681,1.577492,0.633917,18647.084490,0.154302,120941.0,3.850594e+09,...,5.560197,5669.377997,38830.605157,0.597109,0.000925,1527.758592,0.000776,1089.961536,0.003449,10044.747654
13,197,0.018018,0.115348,65.099923,2.054298,0.486784,11734.656758,0.289059,40708.0,1.427557e+10,...,1.277348,874.971523,32660.830673,0.924148,0.026780,685.685963,0.022674,643.348518,0.054633,907.110811
223,96,0.020302,0.146442,74.256313,1.897193,0.527094,12532.114455,0.248245,50570.0,3.235319e+09,...,3.152128,1902.356633,22556.852347,0.707983,0.001566,1837.974760,0.001393,1501.611959,0.003122,5793.888586
103,399,0.031677,0.356531,29.495762,1.410266,0.709086,1879.058324,0.410873,4602.0,6.052421e+08,...,1.674853,146.877110,3143.114116,0.859660,0.004496,833.350822,0.004327,722.615483,0.005436,1704.969828
133,121,0.025970,0.239640,19.949937,1.609954,0.621136,687.207690,0.828710,850.0,1.984541e+08,...,1.138374,20.662759,749.602624,0.957828,0.011637,938.502113,0.011345,899.407513,0.012811,1116.033447


#### Are there NaN values in df_training_radiomics ?

In [33]:
#df_training_radiomics.info()
df_training_radiomics.isnull().sum().sum()

0

### Remark

There are no NaN values in df_training_radiomics.

### Make sure that PatientID are aligned in df_training_clinical_data and df_training_radiomics

In [34]:
(df_training_clinical_data["PatientID"]==df_training_radiomics["PatientID"]).sum()

300

$300$ means that all PatientIDs are aligned in both training dataframes

### Make sure that PatientID are aligned in df_test_clinical_data and df_test_radiomics

In [35]:
(df_test_clinical_data["PatientID"]==df_test_radiomics["PatientID"]).sum()

125

$125$ means that all PatientIDs are aligned in both test dataframes

### Baseline model for survival regression on NSCLC clinical data : Cox proportional hazard (Cox-PH) model

This baseline is trained on a selection of features from both clinical data file and radiomics file. A Cox-PH model was fitted on

- 1 - Tumor sphericity, a measure of the roundness of the shape of the tumor region relative to a sphere, regardless its dimensions (size).
- 2 - The tumor's surface to volume ratio is a measure of the compactness of the tumor, related to its size.
- 3 - The tumor's maximum 3d diameter The biggest diameter measurable from the tumor volume
- 4 - The dataset of origin
- 5 - The N-tumoral stage grading of the tumor describing nearby (regional) lymph nodes involved
- 6 - The tumor's joint entropy, specifying the randomness in the image pixel values
- 7 - The tumor's inverse different, a measure of the local homogeneity of the tumor
- 8 - The tumor's inverse difference moment is another measurement of the local homogeneity of the tumor

### Name of variables

- 1 - original_shape_Sphericity
- 2 - original_shape_SurfaceVolumeRatio
- 3 - original_shape_Maximum3DDiameter
- 4 - l1 (0) or l2 (1)
- 5 - Nstage
- 6 - original_firstorder_Entropy
- 7 - inverse difference (original_glcm_Id)
- 8 - inverse difference moment (original_glcm_Idm) (according to [here](https://static-content.springer.com/esm/art%3A10.1038%2Fncomms5006/MediaObjects/41467_2014_BFncomms5006_MOESM716_ESM.pdf), ctr+F IDMN and [here](https://github.com/cerr/CERR/wiki/GLCM_global_features))

### Remark

Variables used in the baseline use quantitve and qualitive variables. (dataset of origin (l1 or l2)) makes no sens 

In [36]:
df_training_clinical_data.head(5)

,PatientID,Histology,Mstage,Nstage,SourceDataset,Tstage,age
0,202,Adenocarcinoma,0,0,l2,2,66.0000
1,371,large cell,0,2,l1,4,64.5722
2,246,squamous cell carcinoma,0,3,l1,2,66.0452
3,240,nos,0,2,l1,3,59.3566
4,284,squamous cell carcinoma,0,3,l1,4,71.0554


### Encode SourceDataset ("dataset of origin") with value between 0 and n_datasets-1.

In [37]:
encoder = LabelEncoder()
encoder.fit(df_training_clinical_data["SourceDataset"])
df_training_clinical_data["SourceDataset"] = encoder.transform(df_training_clinical_data["SourceDataset"])

In [38]:
df_training_clinical_data.head(5)

,PatientID,Histology,Mstage,Nstage,SourceDataset,Tstage,age
0,202,Adenocarcinoma,0,0,1,2,66.0000
1,371,large cell,0,2,0,4,64.5722
2,246,squamous cell carcinoma,0,3,0,2,66.0452
3,240,nos,0,2,0,3,59.3566
4,284,squamous cell carcinoma,0,3,0,4,71.0554


In [39]:
df_X_train = pd.concat([df_training_radiomics[["original_shape_Sphericity", 
                       "original_shape_SurfaceVolumeRatio", 
                       "original_shape_Maximum3DDiameter",
                       "original_firstorder_Entropy",
                       "original_glcm_Id",
                       "original_glcm_Idm"]],
                        df_training_clinical_data[["SourceDataset",
                                                  "Nstage"]]
                       ], sort=False, axis=1)

In [40]:
df_X_train = df_X_train.astype(float)
df_X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300 entries, 0 to 299
Data columns (total 8 columns):
original_shape_Sphericity            300 non-null float64
original_shape_SurfaceVolumeRatio    300 non-null float64
original_shape_Maximum3DDiameter     300 non-null float64
original_firstorder_Entropy          300 non-null float64
original_glcm_Id                     300 non-null float64
original_glcm_Idm                    300 non-null float64
SourceDataset                        300 non-null float64
Nstage                               300 non-null float64
dtypes: float64(8)
memory usage: 18.9 KB


### AutoML sklearn

In [41]:
automl = autosklearn.regression.AutoSklearnRegressor(time_left_for_this_task=60)
cindex_scorer = autosklearn.metrics.make_scorer(
        name="cindex",
        score_func=cindex,
        optimum=1,
        greater_is_better=True,
        needs_proba=False,
        needs_threshold=False,
    )

start_time = time.time()
automl.fit(df_X_train, df_train_output["SurvivalTime"]) #, metric=cindex_scorer
# Docs for metric in autoML skearln
# https://automl.github.io/auto-sklearn/master/examples/example_metrics.html
# (Metric must be instance of autosklearn.metrics.Scorer.)
execution_time = start_time-time.time()
print("execution_time", execution_time)

/home/damien/.local/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)


Time limit for a single run is higher than total time limit. Capping the limit for a single run to the total time given to SMAC (59.630584)
[WARNING] [2020-01-24 14:54:35,440:AutoMLSMBO(1)::7a74e5472a90e9aa7a8135bb4c77495c] Could not find meta-data directory /home/damien/.local/lib/python3.6/site-packages/autosklearn/metalearning/files/r2_regression_dense
[WARNING] [2020-01-24 14:54:35,452:EnsembleBuilder(1):7a74e5472a90e9aa7a8135bb4c77495c] No models better than random - using Dummy Score!
[WARNING] [2020-01-24 14:54:35,463:EnsembleBuilder(1):7a74e5472a90e9aa7a8135bb4c77495c] No models better than random - using Dummy Score!


/home/damien/.local/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/damien/.local/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/damien/.local/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/damien/.local/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/damien/.local/lib/python3.6/site-packages/autosklearn/evaluation/train_evaluator.py:197: RuntimeWarning: Mean of empty slice
  Y_train_pred = np.nanmean(Y_train_pred_full, axis=0)
/home/damien/.local/lib/python3.6/site-packages/autosklearn/evaluation

[WARNING] [2020-01-24 14:55:25,920:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
[WARNING] [2020-01-24 14:55:25,920:smac.intensification.intensification.Intensifier] Challenger was the same as the current incumbent; Skipping challenger
1
['/tmp/autosklearn_tmp_5756_5424/.auto-sklearn/ensembles/1.0000000000.ensemble', '/tmp/autosklearn_tmp_5756_5424/.auto-sklearn/ensembles/1.0000000001.ensemble', '/tmp/autosklearn_tmp_5756_5424/.auto-sklearn/ensembles/1.0000000002.ensemble', '/tmp/autosklearn_tmp_5756_5424/.auto-sklearn/ensembles/1.0000000003.ensemble', '/tmp/autosklearn_tmp_5756_5424/.auto-sklearn/ensembles/1.0000000004.ensemble', '/tmp/autosklearn_tmp_5756_5424/.auto-sklearn/ensembles/1.0000000005.ensemble', '/tmp/autosklearn_tmp_5756_5424/.auto-sklearn/ensembles/1.0000000006.ensemble', '/tmp/autosklearn_tmp_5756_5424/.auto-sklearn/ensembles/1.0000000007.ensemble', '/tmp/autosklearn_tmp_5756_5424/.auto-sklearn/e

In [42]:
all_information = automl.get_models_with_weights()
index_regressor = 5
weights = []
print("Models used with corresponding weights :\n")
for weight, simple_regression_pipeline in all_information:
    print(str(weight)+" : "+simple_regression_pipeline[index_regressor].choice.__class__.__name__)
    weights.append(weight)
print()
print("sum(weights) = ", np.round(sum(weights),2))

Models used with corresponding weights :

0.7200000000000001 : LibLinear_SVR
0.14000000000000004 : DecisionTree
0.08000000000000002 : LibLinear_SVR
0.060000000000000005 : RidgeRegression

sum(weights) =  1.0


### Metric

In [43]:
automl._automl[0]._metric.name

'r2'

### Test

In [44]:
df_test_clinical_data["SourceDataset"] = encoder.transform(df_test_clinical_data["SourceDataset"])

In [45]:
df_X_test = pd.concat([df_test_radiomics[["original_shape_Sphericity", 
                       "original_shape_SurfaceVolumeRatio", 
                       "original_shape_Maximum3DDiameter",
                       "original_firstorder_Entropy",
                       "original_glcm_Id",
                       "original_glcm_Idm"]],
                        df_test_clinical_data[["SourceDataset",
                                                  "Nstage"]]
                       ], sort=False, axis=1)

In [46]:
df_X_test = df_X_test.astype(float)
df_X_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125 entries, 0 to 124
Data columns (total 8 columns):
original_shape_Sphericity            125 non-null float64
original_shape_SurfaceVolumeRatio    125 non-null float64
original_shape_Maximum3DDiameter     125 non-null float64
original_firstorder_Entropy          125 non-null float64
original_glcm_Id                     125 non-null float64
original_glcm_Idm                    125 non-null float64
SourceDataset                        125 non-null float64
Nstage                               125 non-null float64
dtypes: float64(8)
memory usage: 7.9 KB


In [48]:
y_hat = automl.predict(df_X_test)

In [49]:
df_predicted_survival_time = pd.read_csv(os.path.join(submission_file_path, "random_submission_0vhlEZN.csv"))
df_predicted_survival_time.sample(5)

,PatientID,SurvivalTime,Event
47,322,1538.827887,NaN
70,191,2007.686034,NaN
71,120,1360.147288,NaN
29,125,423.850929,NaN
40,351,904.922744,NaN


In [50]:
df_predicted_survival_time["PatientID"] = df_training_clinical_data["PatientID"]
df_predicted_survival_time["SurvivalTime"] = y_hat

In [53]:
df_predicted_survival_time.sample(5)

,PatientID,SurvivalTime,Event
122,253,1355.305967,NaN
85,103,698.011603,NaN
35,227,672.984124,NaN
57,233,506.537654,NaN
10,372,1553.352324,NaN


## $\color{red}{\text{To be continued}}$